원문: Moler, Cleve B. [Numerical computing with MATLAB](https://www.mathworks.com/moler/chapters.html). Society for Industrial and Applied Mathematics, 2008.

# 5. 최소자승법

*최소자승법*은 과결정 되어 있거나 부정확하게 명세된 연립방정식의 근을 찾는 데 자주 사용되는 방법이다.  엄밀해를 구하는 대신, 우리는 잔차의 제곱의 합을 최소화하는 것 만을 목표로 한다

최소자승 조건은 중요한 통계적 해석을 가진다. 만일 아래에 깔린 오류 분표에 대한 적당한 확률적 가정이 주어지면, 최소자승법은 매개변수의 *최대 가능도* 추정값을 계산해낼 수 있다. 심지어 확률적 가정이 만족되지 않더라도, 다년간의 경험에 따르면, 최소자승법은 유용한 결과값을 계산해 낸다.

최소 자승 문제를 위한 계산 기법은 직교 행렬 분해를 이용한다.

## 5.1 모델과 곡선적합

매우 자주 최소 자승 문제를 접하는 경우는 곡선 적합이다. $t$가 독립변수이고 우리가 추정하고자 하는 미지의 함수를 $y(t)$로 표시한다고 하자. $m$ 개의 관측값이 있다고 가정하라. 이를테면, $t$의 특정 값에서 $y$ 값이 다음과 같이 관측되었다:
$$y_i=y(t_i), i = 1, \cdots, m$$
아이디어는 $y(t)$ 를 $n$ 개의 *기저함수* 의 선형 조합으로 모델하는 것이다.
$$y(t) \approx \beta_1 \phi_1(t)+\cdots+\beta_n \phi_n(t)$$
해당 *설계 행렬* $X$ 는 $m \times n$ 직사각형 행렬로 각 요소는 다음과 같다.
$$x_{i,j}=\phi_j(t_i)$$
설계 행렬은 보통 행이 열보다 많다. 행렬 벡터 표시로 모델은 다음과 같다.
$$y \approx X \beta $$
기호 $\approx$ 는 "대략 비슷하다"는 뜻이다. 우리는 다음 절에서 더 정확한 의미를 논하겠지만, 우리는 중점을 *최소자승* 추정값에 둔다.
기저함수 \phi_j(t) 는 $t$의 비선형함수일 수도 있지만, 미지의 매개변수 $beta_j$ 는 모델에서 선형적으로 나타난다. 연립방정식
$$X\beta\approx y$$
가 *과결정* 되는 조건은 방정식이 미지수 보다 더 많은 경우이다. 예를 들어 다음과 같이 적용할 수 있다.

In [ ]:
# https://stackoverflow.com/questions/7160162/left-matrix-division-and-numpy-solve
    
import numpy as np
import numpy.linalg as na

t0 = np.linspace(0, 10, 11)
t = t0.reshape((len(t0), 1))
a = 0.5
y_true = a * t
noise = np.random.normal(0.0, 0.5, size=y_true.shape)
y_contaminated = y_true + noise
x, resid, rank, s = na.lstsq(t, y_contaminated)
print('x = %s' % x)
print('resid = %s' % resid)
print('rank = %s' % rank)
print('s = %s' % s)

기저함수는 또한 비선형 매개변수 $\alpha_1, \cdots, \alpha_p$ 를 포함하고 있을 수 있다.  문제가 선형과 비선형 매개변수 모두를 포함하면 *분리가능*이라고 말한다:
$$y(t) \approx \beta_1 \phi_1(t, \alpha) + \cdots + \beta_n \phi_n(t, \alpha) $$
설계 행렬의 요소는 $t$ 와 $\alpha$ 모두에 의존한다.
$$x_{i,j} = \phi_j(t_i, \alpha)$$
분리 가능한 문제는 선형 대수적인 방법과 최적화 기법 가운데 한가지를 복합함으로써 풀 수 있다. 
아래 모델을 참고할 수 있다:
* 직선 : 모델이 $t$ 에 대해 선형이라면, 직선이다:
$$y(t) \approx \beta_1 t + \beta_2 $$
* 다항식 : 계수 $\beta_j$ 가 선형적으로 나타난다. 가장 높은 차수의 계수부터 표시한다:
\begin{align}
\phi_j(t) &= t^{n-j}, j=1, \cdots, n \\
y(t) &\approx \beta_1 t^{n-1} + \cdots + \beta_{n-1}t + \beta_n
\end{align}
``polyfit``으로 최소자승 다항식 곡선 적합이 가능한데 설계 행렬을 만들고 선형 대수적으로 매개변수를 푼다.
* 분수 함수 : 분자의 계수는 선형적으로 나타난다; 분모의 계수는 비선형적으로 나타난다:
\begin{align}
\phi_j(t) &= \frac{t^{n-j}}{\alpha_1 t^{n-1} + \cdots + \alpha_{n-1}t + \alpha_n},\\
y(t) &\approx \frac{\beta_1 t^{n-1} + \cdots + \beta_{n-1}t + \beta_n}{\alpha_1 t^{n-1} + \cdots + \alpha_{n-1}t + \alpha_n}
\end{align}
* 지수 함수 : 감쇠율 $\lambda_j$ 가 비선형적으로 나타난다:
\begin{align}
\phi_j(t) &= e^{-\lambda_j t} \\\
y(t) &\approx \beta_1 e^{-\lambda_1 t} + \cdots + \beta_n e^{-\lambda_n t}
\end{align}
* 로그 선형 : 만일 지수가 하나 뿐이라면, 로그를 취하면 모델이 선형으로 만들지만 적합 조건도 바꾼다.
\begin{align}
y(t) &\approx K e^{\lambda t} \\\
log y &\approx \beta_1 t + \beta_2 \\\
\beta_1 &= \lambda \\\
\beta_2 &= log K
\end{align}
* 정규 분포 : 평균과 표준편차가 비선형적으로 나타난다
\begin{align}
\phi_j(t) &= e^{-(\frac{t-\mu_j}{\sigma_j})^2} \\\
y(t) &\approx \beta_1 e^{-(\frac{t-\mu_1}{\sigma_1})^2} + \cdots + \beta_n e^{-(\frac{t-\mu_n}{\sigma_n})^2}
\end{align}

# 5.2 놂

*잔차* 는 관측값과 모델의 차 이다:
$$r_i=y_i - \sum\limits_{j=1}^n \beta_j \phi_j(t_i, \alpha), i = 1, \cdots, m $$
또는 행렬 벡터 표시로는 다음과 같다.
$$r=y-X(\alpha)\beta$$
우리가 찾기를 원하는 것은 잔차를 가능한 작게 만드는 $\alpha$ 와 $\beta$ 이다. "작다"는 것은 어떤 의미인가? 다른 말로는, 우리가 $\approx$ 기호를 사용할 때 어떤 의미로 사용하는가? 몇가지 가능성이 있다.
* 내삽 : 만일 매개변수의 수가 관측값의 수와 같다면, 우리는 잔차를 0으로 만들 수 있을 지도 모른다. 선혐 문제에서는, 이것이 의미하는 것은 $m=n$ 이고 설계 행렬 $X$는 정방행렬이다. 만일 $X$ 가 특이행렬이 아니라면, $\beta$는 정방 연립 방정식의 해 이다:
$$\beta = X \backslash y$$
* 최소자승 : 잔차의 제곱의 합을 최소화한다.
$$||r||^2 = \sum\limits_{i=1}^m {r_i}^2$$
* 가중 최소 자승 : 만일 어떤 관측값이 다른 것에 비해 더 중요하거나 더 정확하다면 우리는 개별 가중치 $w_j$ 와 개별 관측값을 관련짓고 다음을 최소화 할 수 있다.
$${||r||_w}^2 = \sum\limits_{i=1}^m w_i{r_i}^2$$
예를 들어 만일 $i$번째 관측값의 오차가 대략 $e_i$라면, $w_i = 1 / e_i$ 로 한다.
가중치가 적용되지 않은 최소 자승 문제를 푸는 알고리듬이라면 어떤 것이라도 가중치 문제를 푸는데 사용할 수 있다. 우리는 단순히 $y_i$와 $X$의 $i$번째 행에 $w_i$를 곱하기만 하면 된다. 예를 들어 아래와 같은 방식으로 가능하다.

In [ ]:
import numpy as np
import numpy.linalg as na

t0 = np.linspace(0, 10, 11)
t = t0.reshape((len(t0), 1))
a = 0.5
y_true = a * t

noise = np.random.normal(0.0, 0.5,size=t0.shape)
noise[2] = noise[0] = 0.0
noise = noise.reshape(t.shape)

y_contaminated = y_true + noise
x, resid, rank, s = na.lstsq(t, y_contaminated)

print('x = %s' % x)
print('resid = %s' % resid)
print('rank = %s' % rank)
print('s = %s' % s)

w = np.ones_like(t0)
w[2] = w[0] = 10.0
w_diag = np.matrix(np.diag(w))
t_mat = np.matrix(t)
y_c_mat = np.matrix(y_contaminated)

t_weighted_mat = w_diag * t_mat
y_weighted_mat = w_diag * y_c_mat

xw, residw, rankw, sw = na.lstsq(t_weighted_mat, y_weighted_mat)

print('xw = %s' % xw)
print('residw = %s' % residw)
print('rankw = %s' % rankw)
print('sw = %s' % sw)